In [16]:
import pickle

#location of the pickle file written by snippets classifier 
snippets_pkl_loc = ""

#location of the pickle file written by wikifier classifier 
wikifier_pkl_loc = ""

#location of the pickle file written by NER classifier 
ner_pkl_loc = ""

#converts the list of tuples to dict of dicts
def createDictOfFromlistOfTuples(results):
    d = { file_id : { "label" : label, "prob" : prob } for file_id, label , prob in results}
    return d

#merges 3 dicts based on common keys
#s_d : snippets dict
#w_d : wikifier dict
#n_d : NER dict
def merge3dicts(s_d, w_d, n_d):
    s_d_keys = s_d.keys()
    w_d_keys = w_d.keys()
    n_d_keys = n_d.keys()
    
    merged_d = dict()
    
    for key in s_d_keys:
        if key in w_d_keys and key in n_d_keys:
            merged_d.update({ key : { "s" : s_d[key], "w" : w_d[key], "n" : n_d[key] } })
    return merged_d


#load the pickle file : list os tuples
preds_from_snippets = pickle.load(open(snippets_pkl_loc,"rb"))
preds_from_wikifier = pickle.load(open(wikifier_pkl_loc,"rb"))
preds_from_NER = pickle.load(open(ner_pkl_loc,"rb"))

#converting the list of tuples to dict of dicts
preds_from_snippets_dict = createDictOfFromlistOfTuples(preds_from_snippets)
preds_from_wikifier_dict = createDictOfFromlistOfTuples(preds_from_wikifier)
preds_from_ner_dict = createDictOfFromlistOfTuples(preds_from_NER)

#merging all the common predictions made by 3 classifiers
preds_d = merge3dicts( preds_from_snippets_dict, preds_from_wikifier_dict, preds_from_ner_dict)

def makeFinalPrediction( preds_d ):
    #predicted_labels = {}
    """
    if (len(preds_from_snippets) not equals len(preds_from_wikifier)) or \
    (len(preds_from_snippets) not equals len(preds_from_NER)) or 
    (len(preds_from_NER) not equals len(preds_from_wikifier)):
        return
    else:
    """
    #length_of_test_instances = len(preds_from_wikifier)
    #mismatch_pred_count = 0
    for key in preds_d.keys():
        """
        if(( preds_d[key]["s"]["l"] != preds_d[key]["w"]["l"] ) and (preds_d[key]["s"]["l"] != preds_d[key]["n"]["l"])):
            continue
        else:
        """
        #all three classifiers agree on the label
        if preds_d[key]["s"]["l"] == preds_d[key]["w"]["l"]  and preds_d[key]["s"]["l"] == preds_d[key]["n"]["l"]:
            preds_d[key].update( { "f" : { "l" : preds_d[key]["s"]["l"], "p" : 1 } } )
        #snippets and wikifier agree on the label
        elif preds_d[key]["s"]["l"] == preds_d[key]["w"]["l"] :
            preds_d[key].update( { "f" : { "l" : preds_d[key]["s"]["l"], "p" : 2/3 } } )
        #snippets and NER agree on the label
        elif preds_d[key]["s"]["l"] == preds_d[key]["n"]["l"] :
            preds_d[key].update( { "f" : { "l" : preds_d[key]["s"]["l"], "p" : 2/3 } } )
        #NER and wikifier agree on the label
        elif preds_d[key]["w"]["l"] == preds_d[key]["n"]["l"] :
            preds_d[key].update( { "f" : { "l" : preds_d[key]["n"]["l"], "p" : 2/3 } } )
        #None of the three agree on the label, so predict based on the confidence
        else:        
            #snippet is more confident
            if ( preds_d[key]["s"]["p"] > preds_d[key]["w"]["p"] ) and ( preds_d[key]["s"]["p"] > preds_d[key]["n"]["p"] ):
                preds_d[key].update( { "f" : { "l" : preds_d[key]["s"]["l"], "p" : 1/3 } } )
            #ner is more confident
            elif ( preds_d[key]["n"]["p"] > preds_d[key]["s"]["p"] ) and ( preds_d[key]["n"]["p"] > preds_d[key]["w"]["p"] ):
                preds_d[key].update( { "f" : { "l" : preds_d[key]["n"]["l"], "p" : 1/3 } } )
            #wikifier is more confident or all are equally confident of their respective results
            else:
                preds_d[key].update( { "f" : { "l" : preds_d[key]["w"]["l"], "p" : 1/3 } } )
    return preds_d

#finalPreds = makeFinalPrediction( preds_d )
#pickle.dump(finalPreds, open(folderToSaveResultsFile + "/" + 'final.pkl', 'wb'))

In [17]:
"""
d1 = { "1" : {"l" : "arm", "p" : 0.3}, "2" : {"l" : "arch", "p" : 0.4}, "3" : {"l" : "soc", "p" : 0.7}, "4" : {"l" : "hist", "p" : 1} }
d2 = { "1" : {"l" : "psy", "p" : 0.4}, "2" : {"l" : "art", "p" : 0.5}, "3" : {"l" : "chem", "p" : 0.8}}
d3 = { "1" : {"l" : "sci", "p" : 0.5}, "2" : {"l" : "music", "p" : 0.6}, "3" : {"l" : "phy", "p" : 0.9}, "4" : {"l" : "hist", "p" : 0.7}}

def merge3dicts(s_d, w_d, n_d):
    s_d_keys = s_d.keys()
    w_d_keys = w_d.keys()
    n_d_keys = n_d.keys()
    
    merged_d = dict()
    
    for key in s_d_keys:
        if key in w_d_keys and key in n_d_keys:
            merged_d.update({ key : { "s" : s_d[key], "w" : w_d[key], "n" : n_d[key] } })
    return merged_d

m_d = merge3dicts(d1, d2, d3)
print(m_d)
print(len(m_d))
finalPreds = makeFinalPrediction( m_d )
print(finalPreds)
print(len(finalPreds))

for key in finalPreds.keys():
    print("Key : ",key)
    print("Value : ",finalPreds[key])
"""

{'2': {'w': {'l': 'art', 'p': 0.5}, 'n': {'l': 'music', 'p': 0.6}, 's': {'l': 'arch', 'p': 0.4}}, '1': {'w': {'l': 'psy', 'p': 0.4}, 'n': {'l': 'sci', 'p': 0.5}, 's': {'l': 'arm', 'p': 0.3}}, '3': {'w': {'l': 'chem', 'p': 0.8}, 'n': {'l': 'phy', 'p': 0.9}, 's': {'l': 'soc', 'p': 0.7}}}
3
{'2': {'w': {'l': 'art', 'p': 0.5}, 'n': {'l': 'music', 'p': 0.6}, 'f': {'l': 'music', 'p': 0.3333333333333333}, 's': {'l': 'arch', 'p': 0.4}}, '1': {'w': {'l': 'psy', 'p': 0.4}, 'n': {'l': 'sci', 'p': 0.5}, 'f': {'l': 'sci', 'p': 0.3333333333333333}, 's': {'l': 'arm', 'p': 0.3}}, '3': {'w': {'l': 'chem', 'p': 0.8}, 'n': {'l': 'phy', 'p': 0.9}, 'f': {'l': 'phy', 'p': 0.3333333333333333}, 's': {'l': 'soc', 'p': 0.7}}}
3
Key :  2
Value :  {'w': {'l': 'art', 'p': 0.5}, 'n': {'l': 'music', 'p': 0.6}, 'f': {'l': 'music', 'p': 0.3333333333333333}, 's': {'l': 'arch', 'p': 0.4}}
Key :  1
Value :  {'w': {'l': 'psy', 'p': 0.4}, 'n': {'l': 'sci', 'p': 0.5}, 'f': {'l': 'sci', 'p': 0.3333333333333333}, 's': {'l': '